In [ ]:
import os, sys
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.utils.data
import numpy as np
from sklearn.metrics import average_precision_score, precision_recall_curve, accuracy_score
from tqdm import tqdm


base_fold = f"/home/vkocheganov/work/research_projects/dollar/"
sys.path.insert(0, os.path.join(base_fold, "CNNDetection"))
from networks.resnet import resnet50

class OPT: pass
opt = OPT()
# opt.dir = os.path.join(base_fold, 'cnndet_datasets', 'CNN_synth_testset', 'deepfake')

opt.model_path = os.path.join(base_fold, "CNNDetection", "weights", "blur_jpg_prob0.1.pth")
opt.batch_size = 1
opt.workers = 4
opt.crop = None
opt.use_cpu = False
opt.size_only = False

# Load model
if(not opt.size_only):
  model = resnet50(num_classes=1)
  if(opt.model_path is not None):
      state_dict = torch.load(opt.model_path, map_location='cpu')
  model.load_state_dict(state_dict['model'])
  model.eval()
  if(not opt.use_cpu):
      model.cuda()
      
import gc
def infer_dir(directory, model):
  gc.collect()
  opt.dir = directory# os.path.join("/mnt/ssd4tb/vk/dollar/dollar_dataset/")

  # Transform
  trans_init = []
  if(opt.crop is not None):
    trans_init = [transforms.CenterCrop(opt.crop),]
    # print('Cropping to [%i]'%opt.crop)
  # else:
    # print('Not cropping')
  trans = transforms.Compose(
    trans_init + [
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),]
    )

  # Dataset loader
  if(type(opt.dir)==str):
    opt.dir = [opt.dir,]

  # print('Loading [%i] datasets'%len(opt.dir))
  data_loaders = []
  for direct in opt.dir:
    dataset = datasets.ImageFolder(direct, transform=trans)
    data_loaders+=[torch.utils.data.DataLoader(dataset,
                                            batch_size=opt.batch_size,
                                            shuffle=False,
                                            num_workers=opt.workers),]

  y_true, y_pred = [], []
  Hs, Ws = [], []
  with torch.no_grad():
    for data_loader in data_loaders:
      for data, label in tqdm(data_loader[:100]):
      # for data, label in data_loader:

        Hs.append(data.shape[2])
        Ws.append(data.shape[3])

        y_true.extend(label.flatten().tolist())
        if(not opt.size_only):
          if(not opt.use_cpu):
              data = data.cuda()
          y_pred.extend(model(data).sigmoid().flatten().tolist())

  Hs, Ws = np.array(Hs), np.array(Ws)
  y_true, y_pred = np.array(y_true), np.array(y_pred)

  print('Average sizes: [{:2.2f}+/-{:2.2f}] x [{:2.2f}+/-{:2.2f}] = [{:2.2f}+/-{:2.2f} Mpix]'.format(np.mean(Hs), np.std(Hs), np.mean(Ws), np.std(Ws), np.mean(Hs*Ws)/1e6, np.std(Hs*Ws)/1e6))
  print('Num reals: {}, Num fakes: {}'.format(np.sum(1-y_true), np.sum(y_true)))

  if(not opt.size_only):
    r_acc = accuracy_score(y_true[y_true==0], y_pred[y_true==0] > 0.5)
    f_acc = accuracy_score(y_true[y_true==1], y_pred[y_true==1] > 0.5)
    acc = accuracy_score(y_true, y_pred > 0.5)
    print(type(y_true), y_true)
    print(type(y_pred), y_pred)
    ap = average_precision_score(y_true, y_pred)

    print('AP: {:2.2f}, Acc: {:2.2f}, Acc (real): {:2.2f}, Acc (fake): {:2.2f}'.format(ap*100., acc*100., r_acc*100., f_acc*100.))

dir_to_infer = "/home/vkocheganov/work/research_projects/dollar/cnndet_datasets/CNN_synth_testset/"
for curr_dir in os.listdir(dir_to_infer):
  if curr_dir != "progan":
    continue
  print(f"processing {curr_dir}")
  infer_dir(os.path.join(dir_to_infer, curr_dir), model)

print(f"processing test")
infer_dir("/mnt/ssd4tb/vk/dollar/dollar_dataset/", model)

/tmp/ipykernel_3693265/1276502182.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(opt.model_path, map_location='cpu')


processing progan


100%|██████████| 8000/8000 [00:48<00:00, 165.31it/s]


Average sizes: [256.00+/-0.00] x [256.00+/-0.00] = [0.07+/-0.00 Mpix]
Num reals: -68000, Num fakes: 76000


ValueError: Expected 2D array, got 1D array instead:
array=[8.68100480e-10 1.42288514e-10 1.03142118e-14 ... 1.00000000e+00
 1.00000000e+00 1.00000000e+00].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
# /tmp/ipykernel_3693265/638286014.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
#   state_dict = torch.load(opt.model_path, map_location='cpu')
# processing gaugan
# 100%|██████████| 10000/10000 [01:01<00:00, 162.45it/s]
# Average sizes: [256.00+/-0.00] x [256.00+/-0.00] = [0.07+/-0.00 Mpix]
# Num reals: 5000, Num fakes: 5000
# AP: 90.80, Acc: 81.42, Acc (real): 92.98, Acc (fake): 69.86
# processing whichfaceisreal
# 100%|██████████| 2000/2000 [01:07<00:00, 29.82it/s]
# Average sizes: [1024.00+/-0.00] x [1024.00+/-0.00] = [1.05+/-0.00 Mpix]
# Num reals: 1000, Num fakes: 1000
# AP: 99.91, Acc: 95.10, Acc (real): 99.80, Acc (fake): 90.40
# processing seeingdark
# 100%|██████████| 360/360 [02:56<00:00,  2.04it/s]
# Average sizes: [3420.27+/-591.67] x [5113.43+/-886.51] = [18.01+/-6.09 Mpix]
# Num reals: 180, Num fakes: 180
# AP: 99.81, Acc: 98.06, Acc (real): 97.78, Acc (fake): 98.33
# processing crn
# 100%|██████████| 12764/12764 [01:29<00:00, 143.19it/s]
# Average sizes: [256.00+/-0.00] x [512.00+/-0.00] = [0.13+/-0.00 Mpix]
# Num reals: 6382, Num fakes: 6382
# AP: 99.84, Acc: 86.35, Acc (real): 72.70, Acc (fake): 100.00
# processing san
# 100%|██████████| 438/438 [00:07<00:00, 56.96it/s] 
# Average sizes: [563.13+/-244.18] x [690.60+/-287.70] = [0.44+/-0.32 Mpix]
# Num reals: 219, Num fakes: 219
# AP: 68.57, Acc: 50.00, Acc (real): 99.54, Acc (fake): 0.46
# processing progan
# 100%|██████████| 8000/8000 [00:49<00:00, 161.28it/s]
# Average sizes: [256.00+/-0.00] x [256.00+/-0.00] = [0.07+/-0.00 Mpix]
# Num reals: -68000, Num fakes: 76000
